In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from collections import defaultdict
import gzip
import numpy as np
from tqdm import tqdm

# one-hot encoder

def encode_one_hot(train_set, classes="AGCT"):
    encoder = defaultdict(lambda: np.array([0]*len(classes)))
    pad = 9
        
    for i, _class in enumerate(classes):
        tmp = np.zeros(len(classes))
        tmp[i] = 1
        encoder[_class] = tmp
        
    output = []
    for record in train_set:
        encoded_record = []
        
    output = []
    for record in train_set:
        encoded_record = []
        for c in record.upper():
            encoded_record.append(encoder[c])
            
        output.append(encoded_record)
        
    output = np.array(output)
    
    return output

In [ ]:
# Parsing
import gzip
from tqdm import tqdm
from sklearn.model_selection import train_test_split

# single dataset version
def get_dataset(path):
    dataset =  gzip.open(path, "r")
    output = []    
    for i, record in tqdm(enumerate(dataset)):
        record = record.decode()
        record = record[:-1]
        record = record.split('\t')
        if i != 0:
            output.append(record)
    output = np.array(output).reshape([-1, 4])
    
    return encode_one_hot(output[:, 2]), np.array([int(i) for i in output[:, 3]]).flatten()

x1, y1 = get_dataset("/content/drive/Shareddrives/GP/Aptamer/data/DeepBind/Alx1_DBD_TAAAGC20NCG_3_Z_A.seq.gz")
x2, y2 = get_dataset("/content/drive/Shareddrives/GP/Aptamer/data/DeepBind/Alx1_DBD_TAAAGC20NCG_3_Z_B.seq.gz")
x = np.append(x1, x2, axis=0)
y = np.append(y1, y2, axis=0)

128013it [00:00, 132092.71it/s]
255509it [00:01, 186575.89it/s]


In [ ]:
# 데이터셋 이름 변경 (valid -> test), validation을 진행 안하네?
train_x, test_x, train_y, test_y = train_test_split(x, y, test_size = 0.2, random_state=112)

# index 에러가 떠서 수정
train_x = train_x.reshape(-1,80)
test_x = test_x.reshape(-1,80)

print(train_x.shape)
print(train_y.shape)

print(test_x.shape)
print(test_y.shape)

del(y)

(306816, 80)
(306816,)
(76704, 80)
(76704,)


In [ ]:
#RandomForest
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics
from sklearn.metrics import classification_report, roc_auc_score
 
forest = RandomForestClassifier(n_estimators=100) #랜덤포레스트 모델 생성
forest.fit(train_x, train_y)  #모델 학습
 
# 변수이름변경에 따른 변경 (valid -> test)
y_pred = forest.predict(test_x)  #모델 측정
score = classification_report(test_y, y_pred, digits=5)  #모델 검사
print(score)
print(f'AUC : {roc_auc_score(test_y, y_pred)}')


              precision    recall  f1-score   support

           0    0.84782   0.85672   0.85225     25544
           1    0.92808   0.92322   0.92565     51160

    accuracy                        0.90107     76704
   macro avg    0.88795   0.88997   0.88895     76704
weighted avg    0.90135   0.90107   0.90120     76704

AUC : 0.8899695434231498


In [ ]:
#onehot_decoder
def onehot_decoder(seq, word=['A','G','C','T']):
    out = ""
    
    for i in range(9, 29):
        arg = np.argmax(seq[i])
        out += word[arg]
    #out+="/"
        
    return out

In [ ]:
#Genetic Algorithm
 
#Initialize
def init():
  pool = []
  for i in range(30):   #유전자풀은 30개, 랜덤으로 초기화함
    seq = []
    for i in range(9):
        seq.append([.25, .25, .25, .25])
    for i in range(20):
        acid = [0, 0, 0, 0]
        n = np.random.randint(0,4)
        acid[n] = 1
        seq.append(acid)
    for i in range(9):
        seq.append([.25, .25, .25, .25])
    pool.append(seq)
  return np.array(pool)
 
#fitness
def fitness(model, pool):
  scores = [0]*30
  for i in range(30):
    proba = model.predict_proba([pool[i][9:29].ravel()])
    scores[i] = proba[0,1]
  
  return scores
 
#selection
def selection(pool, scores):
  temp = []
  while len(temp) < 8:  #적합도가 가장 높은 8개의 유전자를 선정
    index = np.argmax(scores)
    if pool[index] not in temp:
      temp.append(pool[index])
    del scores[index]
    del pool[index]
 
  p1 = temp[0][:] #4개의 부모 유전자
  p2 = temp[1][:]
  p3 = temp[2][:]
  p4 = temp[3][:]
  x = np.random.randint(9,29) #교차 위치 선정
  c1 = p1[:x] + p2[x:] #4개의 자식 유전자 생성
  c2 = p2[:x] + p1[x:]
  x = np.random.randint(9,29)
  c3 = p3[:x] + p4[x:]
  c4 = p4[:x] + p3[x:]
  temp.append(c1)
  temp.append(c2)
  temp.append(c3)
  temp.append(c4)
 
  n = np.random.randint(10,16)  #부모 및 자식의 돌연변이 갯수 선정
  for i in range(n):
    index = np.random.randint(0,12)  #오리지날 부모 및 자식 중 하나 선택
    temp_seq = temp[index][:]
    pos_a = np.random.randint(9,29) #돌연변이 시작위치 선정
    pos_b = np.random.randint(pos_a,29) #돌연변이 끝 위치 선정
    for j in range(pos_a, pos_b+1): #pos_a 부터 pos_b까지의 모든 아미노산을 돌연변이 시킴
      temp_acid = [0, 0, 0, 0]
      r = np.random.randint(0,4)  #돌연변이 값 (A,G,T,C 중 하나) 선정
      temp_acid[r] = 1
      temp_seq[j] = temp_acid[:]
 
    temp.append(temp_seq)
 
  for i in range(18-n): #부모와 자식, 돌연변이 외 나머지는 완전 무작위로 sequence를 생성
    seq = []
    for i in range(9):
        seq.append([.25, .25, .25, .25])
    for i in range(20):
        acid = [0, 0, 0, 0]
        n = np.random.randint(0,4)
        acid[n] = 1
        seq.append(acid)
    for i in range(9):
        seq.append([.25, .25, .25, .25])

    temp.append(seq)
 
  return np.array(temp)

In [ ]:
import tensorflow as tf
import operator
# load model
deepbind = tf.keras.models.load_model("/content/CNN_Final_Alx1.h5")

In [ ]:
def scoring(model):
  deepScores = []
  for i in tqdm(range(20)):
    pool = init()
    for j in range(50):
      scores = fitness(model, pool)
      pool = selection(pool.tolist(), scores)[:]
    
    scores = fitness(model, pool)
    index = np.argmax(scores)
    most_seq = pool[index]
    decode_seq = onehot_decoder(most_seq)
    deep_score = deepbind.predict(most_seq.reshape(1,38,4))[0][0]
    print(f'max score sequence : {decode_seq}, deepbind score : {deep_score}')
    deepScores.append([decode_seq, deep_score])

  return deepScores

In [ ]:
deepScores = scoring(forest)

  5%|▌         | 1/20 [00:14<04:42, 14.88s/it]

max score sequence : AATTAAGTAGATCAGACCGT, deepbind score : 0.9833029508590698


 10%|█         | 2/20 [00:29<04:26, 14.82s/it]

max score sequence : GAACAACGGGCAGCGCCACC, deepbind score : 0.7191342115402222


 15%|█▌        | 3/20 [00:44<04:13, 14.92s/it]

...

max score sequence : GCACCTAATCACATTAACGC, deepbind score : 0.9997291564941406


 95%|█████████▌| 19/20 [04:45<00:14, 14.81s/it]

max score sequence : AGTTAATTACATTAGTGCAC, deepbind score : 0.9998548626899719


100%|██████████| 20/20 [04:59<00:00, 14.99s/it]

max score sequence : GAGTTAATTTAATTATCACA, deepbind score : 0.9998329877853394


In [ ]:
import operator
ordered_Scores = sorted(deepScores, key=operator.itemgetter(1), reverse=True)
print(ordered_Scores)


[['CGAGTAATCCAATTAGAGCT', 0.9999764], ['CACGCTAACTAATTAAATTA', 0.9999536], ['CACCTAATTCGATTAGTGGG', 0.99991393], ['AAGTTAATTCAATTAGATGA', 0.99990964], ['GCGCGTAATTCGATTAGCGT', 0.9998932], ['AGTTAATTACATTAGTGCAC', 0.99985486], ['GAGTTAATTTAATTATCACA', 0.999833], ['AATTAAGCCAATTAGCGGAT', 0.9998022], ['GCACCTAATCACATTAACGC', 0.99972916], ['CTCATGTAATTAGATTGTCG', 0.9996488], ['ATAACTTAATTAGATTCGCA', 0.999646], ['TCATTAACTGAATTAGCGAC', 0.9995042], ['TTAACCTAATTATCGTACGG', 0.9992204], ['TGCTCAGCCTAATCACATTA', 0.99903667], ['TTAAGTTAATTAGCCCAGGT', 0.99831903], ['TCCTAATCGAGTTACCGCAT', 0.99769086], ['TAATTAGTGCACCTAATTAA', 0.9948976], ['AATTAAGTAGATCAGACCGT', 0.98330295], ['AATTAACGGTCTGCGCCCCA', 0.9348242], ['GAACAACGGGCAGCGCCACC', 0.7191342]]


In [ ]:
f = open("/content/RandomForest_ScoreResult_Alx1.txt", 'w')
for seq, score in ordered_Scores:
  f.write(f'{seq} {score}\n')
f.close()